In [1]:
#!/usr/bin/env python3
import os
os.environ["DIAMBRAROMSPATH"] = os.getcwd() 
os.environ["DIAMBRA_ENVS"] = """0.0.0.:50001"""
import wandb
from wandb.integration.sb3 import WandbCallback
from diambra.arena.stable_baselines3.make_sb3_env import make_sb3_env, EnvironmentSettings
from diambra.arena import EnvironmentSettings, WrappersSettings
from stable_baselines3 import DQN
# settings = EnvironmentSettings()
# settings.frame_shape = (84,84,1)
# # Wrapper settings specification
# wrappers_settings = WrappersSettings()
# wrappers_settings.stack_frames = 4
# wrappers_settings.dilation = 1
# wrappers_settings.normalize_reward = True
# wrappers_settings.normalization_factor = 0.5
# env = diambra.arena.make("sfiii3n", settings, wrappers_settings)

def collect_attrs(x):
    t = {i :getattr(x,i) for i in dir(x) if i[0] != '_'}
    t = {i:t[i] for i in t if not hasattr(t[i], '__call__')}
    return t  

def main():

    print("Activated {} environment(s)".format(num_envs))
    print("\nStarting training ...\n")
    config = {
        "env" : collect_attrs(settings), 
        "wrapper" : collect_attrs(wrappers_settings),
        "algorithm" : {"type" : "dqn", "model" : {"type" : "MultiInputPolicy"}}}
    run = wandb.init(
        name="dqn",
        project="project4",
        config=config,
        sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
        monitor_gym=True,  # auto-upload the videos of agents playing the game
        save_code=False,  # optional
    )
    agent = DQN("MultiInputPolicy", env, verbose=1, tensorboard_log=f"runs/{run.id}")
    agent.learn(total_timesteps=2000000, callback=WandbCallback(
        gradient_save_freq=100,
        model_save_path=f"models/{run.id}",
        verbose=2))
    agent.save("MultiInputPolicy-DQN")
    print("\n .. training completed.")
    
    print("\nStarting trained agent execution ...\n")
    observation = env.reset()
    while True:
        env.render()

        action, _state = agent.predict(observation, deterministic=True)
        observation, reward, done, info = env.step(action)

        if done:
            observation = env.reset()
            break
    print("\n... trained agent execution completed.\n")

    # Close the environment
    env.close()

    # Return success
    return 0

# if __name__ == "__main__":
#     env, num_envs = main()

In [6]:
import diambra
def build_env(no_resize:bool=False, sb3:bool = True, render_mode=None):
        # Settings
    from diambra.arena import SpaceTypes

    settings = EnvironmentSettings()
    if not no_resize:
        settings.frame_shape = (128, 128, 1)
    settings.characters = ("Alex")
    settings.difficulty = 8
    settings.action_space = SpaceTypes.DISCRETE

    # Wrappers Settings
    wrappers_settings = WrappersSettings()
    wrappers_settings.normalize_reward = True
    wrappers_settings.stack_frames = 5
    wrappers_settings.add_last_action = True
    wrappers_settings.stack_actions = 12
    wrappers_settings.scale = True
    wrappers_settings.exclude_image_scaling = True
    wrappers_settings.role_relative = True
    wrappers_settings.flatten = True
    wrappers_settings.filter_keys = ["action", "own_health", "opp_health", "own_side", "opp_side", "opp_character", "stage", "timer"]

    config = {"policy_type": "MultiInputPolicy"}


    # Create environment
    if sb3:
        env, _ = make_sb3_env("sfiii3n", settings, wrappers_settings, render_mode=render_mode)
    else:
        env = diambra.arena.make("sfiii3n", settings, wrappers_settings, render_mode=render_mode)
    return env

In [32]:
agent = DQN.load("models/4u72x3ey/model.zip")

/home/nchen3/miniconda3/envs/rl/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:437: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  th_object = th.load(fil

In [132]:
env = build_env(no_resize=True, sb3=False, render_mode="human")

INFO:diambra.arena.engine.interface:Trying to connect to DIAMBRA Engine server (timeout=600s)...
INFO:diambra.arena.engine.interface:... done.
INFO:diambra.arena.arena_gym:EnvironmentSettings(game_id='sfiii3n', frame_shape=(0, 0, 0), step_ratio=6, disable_keyboard=True, disable_joystick=True, render_mode='human', splash_screen=True, rank=0, env_address='0.0.0.0:50051', grpc_timeout=600, seed=1732044712, difficulty=8, continue_game=0.0, show_final=False, tower=3, _last_seed=1732044712, pb_model=game_id: "sfiii3n"
frame_shape {
}
step_ratio: 6
n_players: 1
disable_keyboard: true
disable_joystick: true
action_spaces: DISCRETE
episode_settings {
}
, n_players=1, action_space=1, role=None, characters='Alex', outfits=1, super_art=None, fighting_style=None, ultimate_style=None, speed_mode=None)


In [62]:
import numpy as np
for i in obs_sb3.keys():
    t = obs_sb3[i] != obs_native[i]
    if isinstance(t, np.ndarray): 
        t = np.any(t)
    if t:
        print(i)
        print(obs_sb3[i])
        print(obs_native[i])
from collections import OrderedDict
def convert2order(d):
    k = list(d.keys())
    k.sort()
    result = OrderedDict()
    for i in k:
        result[i] = d[i]
    return result
convert2order(obs_native)

OrderedDict([('action',
              array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)),
             ('frame',
              array([[[ 38,  38,  38,  38,  38],
                      [ 41,  4

In [103]:
env = build_env(no_resize=False, sb3=True, render_mode="human")

INFO:diambra.arena.engine.interface:Trying to connect to DIAMBRA Engine server (timeout=600s)...
INFO:diambra.arena.engine.interface:... done.
INFO:diambra.arena.arena_gym:EnvironmentSettings(game_id='sfiii3n', frame_shape=(128, 128, 1), step_ratio=6, disable_keyboard=True, disable_joystick=True, render_mode='human', splash_screen=True, rank=0, env_address='0.0.0.0:50051', grpc_timeout=600, seed=1732043442, difficulty=None, continue_game=0.0, show_final=False, tower=3, _last_seed=1732043442, pb_model=game_id: "sfiii3n"
frame_shape {
  h: 128
  w: 128
  c: 1
}
step_ratio: 6
n_players: 1
disable_keyboard: true
disable_joystick: true
action_spaces: DISCRETE
episode_settings {
}
, n_players=1, action_space=1, role=None, characters='Alex', outfits=1, super_art=None, fighting_style=None, ultimate_style=None, speed_mode=None)


In [130]:
env.close()

In [125]:
import cv2
def preprocess(obs, width=128, height=128):
    frames = obs["frame"]
    fs = frames.shape[2]
    result = []
    for i in range(0, fs, 3):
        frame = frames[:,:,i:(i+3)]
        # apply grayscaled first, then resize
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (width, height), interpolation=cv2.INTER_LINEAR)[:, :, None]
        result.append(frame.squeeze(-1))
    frames = np.stack(result, axis=-1)
    obs["frame"] = frames
    return frames

In [139]:

def test_agent(env, agent, seed=42):
    # Environment reset
    observation = env.reset()[0]
    # Agent-Environment interaction loop
    while True:
        preprocess(observation)
        observation = convert2order(observation)
        # (Optional) Environment rendering
        env.render()
        
        # Action random sampling
        actions, _state = agent.predict(observation, deterministic=True)
        # Environment stepping
        observation, reward, terminated, truncated, info = env.step(int(actions))
        if reward != 0:
            print(reward)
        # Episode end (Done condition) check
        if terminated or truncated:
            observation = env.reset()
            break

    # # Environment shutdown
    # env.close()

    # Return success
    return 0
test_agent(env, agent, seed=42)

0.2360248447204969
0.2360248447204969
0.16149068322981366
-0.024844720496894408
0.2360248447204969
0.2360248447204969
-0.024844720496894408
0.16149068322981366
0.2360248447204969
0.2236024844720497
0.13664596273291926
0.049689440993788817
0.2360248447204969
0.2360248447204969
-0.024844720496894408
0.16149068322981366
0.2360248447204969
0.2360248447204969
-0.049689440993788817
-0.2981366459627329
0.2360248447204969
0.2360248447204969
0.2236024844720497
0.18633540372670807
0.012422360248447204
0.13664596273291926
-0.09937888198757763
0.2111801242236025
-0.024844720496894408
-0.09937888198757763
0.2111801242236025
-0.2111801242236025
0.13664596273291926
-0.09937888198757763
-0.049689440993788817
-0.037267080745341616
-0.037267080745341616
-0.024844720496894408
-0.024844720496894408
-0.012422360248447204
0.2111801242236025
0.2111801242236025
0.13664596273291926
0.13664596273291926
0.2608695652173913
0.18633540372670807
0.14906832298136646
-0.037267080745341616
-0.024844720496894408
-0.0993

0

In [137]:
def test_agent(env, agent, seed=42):

    # Environment reset
    observation = env.reset()

    # Agent-Environment interaction loop
    while True:
        # (Optional) Environment rendering
        env.render()

        # Action random sampling
        actions, _state = agent.predict(observation, deterministic=True)

        # Environment stepping
        observation, reward, done, info = env.step(actions)
        if reward[0] != 0:
            print(reward[0])
        # Episode end (Done condition) check
        if done:
            observation = env.reset()
            break

    # # Environment shutdown
    # env.close()

    # Return success
    return 0
test_agent(env, agent, seed=42)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [28]:
env.render.__dir__()

['__new__',
 '__repr__',
 '__hash__',
 '__call__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__reduce__',
 '__func__',
 '__self__',
 '__doc__',
 '__str__',
 '__init__',
 '__reduce_ex__',
 '__getstate__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [ ]:

    
import diambra.arena
env = diambra.arena.make("sfiii3n", render_mode="human")

# mkdir ~/.diambra
# touch ~/.diambra/credentials

# apptainer run --rm --name engine \
#   -v $HOME/.diambra/credentials:/tmp/.diambra/credentials \
#   -v $PWD:/opt/diambraArena/roms \
#   --net=host docker.io/diambra/engine:latest

def main():
    # Environment creation
    env = diambra.arena.make("sfiii3n", render_mode="human")

    # Environment reset
    observation, info = env.reset(seed=42)

    # Agent-Environment interaction loop
    while True:
        # (Optional) Environment rendering
        env.render()

        # Action random sampling
        actions = env.action_space.sample()

        # Environment stepping
        observation, reward, terminated, truncated, info = env.step(actions)
        print(reward)
        # Episode end (Done condition) check
        if terminated or truncated:
            observation, info = env.reset()
            break

    # Environment shutdown
    env.close()

    # Return success
    return 0

if __name__ == '__main__':
    main()

In [ ]:
env.close()

In [ ]:
from stable_baselines3 import DQN
model = DQN("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=10000, log_interval=4)
model.save("dqn_cartpole")

In [ ]:
x,y = env.reset()
x['frame'].shape

In [ ]:
def make_sb3_env(game_id: str, env_settings: EnvironmentSettings=EnvironmentSettings(),
                 wrappers_settings: WrappersSettings=WrappersSettings(),
                 episode_recording_settings: RecordingSettings=RecordingSettings(),
                 render_mode: str="rgb_array", seed: int=None, start_index: int=0,
                 allow_early_resets: bool=True, start_method: str=None, no_vec: bool=False,
                 use_subprocess: bool=True, log_dir_base: str="/tmp/DIAMBRALog/"):


In [ ]:
from diambra.arena import EnvironmentSettings, WrappersSettings
settings = EnvironmentSettings()
settings.frame_shape = (84,84,1)
# Wrapper settings specification
wrappers_settings = WrappersSettings()
wrappers_settings.stack_frames = 4
wrappers_settings.dilation = 1
wrappers_settings.normalize_reward = True
wrappers_settings.normalization_factor = 0.5
env = diambra.arena.make("sfiii3n", settings, wrappers_settings)

In [ ]:
DQN("x", env, verbose=1)

In [ ]:
DQN.policy_aliases